In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# 가상의 데이터 생성
num_users = 1000
num_months = 12

# 데이터프레임 생성
index = [f"user_{i}" for i in range(num_users)]
columns = [f"{month}월" for month in range(1, num_months+1)]

# Generate data with upward trend for passing and downward trend for failing
passing_data = np.random.randint(50, 101, size=(num_users, num_months))
failing_data = np.random.randint(0, 51, size=(num_users, num_months))

# 합격 불합격 여부 라벨값 생성 (가상의 값)
labels = np.random.randint(0, 2, size=num_users)

# Reshape the labels to match the shape of data arrays
reshaped_labels = labels.reshape(-1, 1)

# Combine passing and failing data
data = np.where(reshaped_labels == 1, passing_data, failing_data)

# 데이터프레임 생성
df = pd.DataFrame(data, index=index, columns=columns)
df['Label'] = labels
print(df)
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)  # Remove this line
        
        return out
    
    
    
# Convert the DataFrame to PyTorch tensors
data_tensor = torch.tensor(df.drop(columns=['Label']).values, dtype=torch.float32)
label_tensor = torch.tensor(df['Label'].values, dtype=torch.float32).view(-1, 1)



# Split the data into training and testing sets
train_size = int(0.8 * len(data_tensor))
train_data = data_tensor[:train_size]
train_labels = label_tensor[:train_size]
test_data = data_tensor[train_size:]
test_labels = label_tensor[train_size:]

# Create an instance of the LSTMModel
input_size = num_months
hidden_size = 64
num_layers = 2
output_size = 1

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
        
        
model.eval()
with torch.no_grad():
    test_outputs = model(test_data)
    test_preds = torch.sigmoid(test_outputs).round()

accuracy = (test_preds == test_labels).float().mean()
print(f'Test Accuracy: {accuracy.item():.4f}')




           1월   2월  3월  4월   5월  6월  7월  8월  9월  10월  11월  12월  Label
user_0     88   78  64  92   57  70  88  68  72   60   60   73      1
user_1     85   89  73  52   71  51  73  93  79   87   51   70      1
user_2     82   61  71  93   74  98  76  91  77   65   64   96      1
user_3    100   93  52  86  100  56  70  58  88   67   53   74      1
user_4     63   99  58  75   51  69  77  96  56   93   57   96      1
...       ...  ...  ..  ..  ...  ..  ..  ..  ..  ...  ...  ...    ...
user_995   91  100  65  73   99  63  52  83  82   99   73   58      1
user_996   35    7  49   3   42  15   2   6   1    0    0   37      0
user_997   70   88  69  57   91  86  58  74  74   82   51   68      1
user_998   95   69  91  90   91  78  55  66  74   87   53   79      1
user_999    9   13  38  35   33  23  17  15  40   36   40   24      0

[1000 rows x 13 columns]
Epoch [10/50], Loss: 0.6573
Epoch [20/50], Loss: 0.5819
Epoch [30/50], Loss: 0.4480
Epoch [40/50], Loss: 0.2749
Epoch [50/50], Loss: 0

In [12]:
# Function to predict pass or fail based on a single input
def predict_single_input(input_data):
    model.eval()
    print(input_data)
    with torch.no_grad():
        input_tensor = torch.tensor(input_data, dtype=torch.float32).view(1, -1)
        output = model(input_tensor)
        prediction = torch.sigmoid(output).round().item()
    print(prediction)
    return "Pass" if prediction == 1 else "Fail"

# Example usage
input_data = [ 95  , 69,  91,  90 ,  91,  78 , 55 , 66,  74 , 0 ,  0   ,0 ]  # Replace with your input data
#예측할때입력 데이터 이후에는 평균으로 넣어서 예측하자
result = predict_single_input(input_data)
print(f"Prediction: {result}")

[95, 69, 91, 90, 91, 78, 55, 66, 74, 0, 0, 0]
0.0
Prediction: Fail
